In [39]:
# importing libraries/dependencies

import selenium
import pandas as pd
import numpy as np
import time
import requests
import re
import warnings 
warnings.filterwarnings('ignore')

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementNotVisibleException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, ElementClickInterceptedException

In [40]:
# connecting to the webdriver
driver = webdriver.Chrome("chromedriver.exe")
driver.maximize_window()

In [41]:
# getting the webpage of mentioned url
url = "https://flight.yatra.com/air-search-ui/dom2/trigger?type=O&viewName=normal&flexi=0&noOfSegments=1&origin=DEL&originCountry=IN&destination=BOM&destinationCountry=IN&flight_depart_date=25%2F07%2F2022&ADT=1&CHD=0&INF=0&class=Economy&source=fresco-home&unqvaldesktop=1166763500038"
driver.get(url)
time.sleep(3)
print("Sucessfully webpage open")

Sucessfully webpage open


In [42]:
# let's make a list of locations to scrape data
loc_lst = ['New Delhi', 'Mumbai', 'Bangalore', 'Chennai', 'Hyderabad', 'Goa', 'Kolkata', 'Jaipur', 'Lucknow']

In [43]:
# creating empty lists to fetch required data

Airline_name = []   
Departure_time = []     
Arrival_time = []       
Duration = []           
Stops = []              
Source = []             
Destination = []        
Meal = []               
Price = []              
Location = []          

In [44]:
# scraping the data from the mentioned url
for x in loc_lst:
    for y in loc_lst:
        if x!=y:
            Source = driver.find_element_by_xpath("//div[@class='input-holder pb-2 bdr-btm']/input")
            time.sleep(3)
            Source.clear()
            Source.send_keys(x)
            time.sleep(2)
            
            # fetching webelement of destination
            Destination = driver.find_element_by_xpath("//div[@class='input-holder  bdr-btm pb-2']/input")
            Destination.click()
            Destination.clear()
            time.sleep(2)
            Destination.send_keys(y)
            time.sleep(2)
            
            try:
                driver.find_element_by_xpath("/html/body/section[2]/section/div[1]/div/div[1]").click()
            except NoSuchElementException:
                pass
            
            # search for flights again and clicking in search again button
            try:
                srch_btn = driver.find_element_by_xpath("//button[@class='fs-14 btn-submit cursor-pointer bold']")
                wait = WebDriverWait(driver, 10)
                wait.until(EC.visibility_of(srch_btn))
                srch_btn.click()
            except NoSuchElementException:
                pass
            time.sleep(2)
            
            # scrolling the data 5000 times
            for _ in range(15):
                time.sleep(1)
                driver.execute_script("window.scrollBy(0,5000)")
                time.sleep(4)
                
            # fetching web element of scroll to top button
            try:
                driver.find_element_by_xpath("//div[@title='scroll to top']").click()
            except NoSuchElementException:
                pass
            
            # Fetching name of Airline
            try:
                for i in driver.find_elements_by_xpath("//div[@class='fs-13 airline-name no-pad col-8']/span"):
                    Airline_name.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching departure time of the flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pr']"):
                    Departure_time.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching arrival time of the flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b pdd-0 text-left atime col-5']//p[1]"):
                    Arrival_time.append(i.get_attribute("innerHTML").splitlines()[0][0:5])
            except NoSuchElementException:
                pass
            
            # fteching duration of flights journey
            try:
                for i in driver.find_elements_by_xpath("//div[@class='stop-cont pl-13']/p"):
                    Duration.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching number of stops the flights have between source and destination
            try:
                for i in driver.find_elements_by_xpath("//div[@class=' font-lightgrey fs-10 tipsy i-b fs-10']/span[1]"):
                    Stops.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching availability of meal in flights
            try:
                for i in driver.find_elements_by_xpath("//div[@class='features pull-right fs-12 flex']/div[1]"):
                    Meal.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching locations
            try:
                for i in driver.find_elements_by_xpath("//p[@class='fs-10 font-lightgrey no-wrap city ellipsis']"):
                    Location.append(i.text)
            except NoSuchElementException:
                pass
            
            # fetching prices fo flight tickets
            try:
                for i in driver.find_elements_by_xpath("//div[@class='i-b tipsy fare-summary-tooltip fs-18']"):
                    Price.append(i.text)
            except NoSuchElementException:
                pass


print("Copmpleted")

Copmpleted


In [45]:
Source = Location[0:len(Location):2]
Destination = Location[1:len(Location):2]

In [46]:
# checking lengths of all features
print(len(Price), len(Airline_name), len(Departure_time), len(Arrival_time), len(Duration), len(Stops), len(Meal))


3355 3355 3355 3355 3355 3355 3355


In [47]:
# creating a dataframe for scraped data
data = list(zip(Airline_name,Departure_time,Arrival_time,Duration,Source,Destination,Meal,Stops,Price))
df = pd.DataFrame(data, columns =["Airline",
                                  "Departure_time",  
                                  "Time_of_arrival",      
                                  "Duration",                                        
                                  "Source",        
                                  "Destination",
                                  "Meal_availability",
                                  "Number_of_stops",
                                  "Price"])
df

,Airline,Departure_time,Time_of_arrival,Duration,Source,Destination,Meal_availability,Number_of_stops,Price
0,Vistara,14:20:00,16:30:00,2h 10m,New Delhi,Mumbai,No Meal Fare,Non Stop,5955
1,Vistara,15:30:00,17:40:00,2h 10m,New Delhi,Mumbai,No Meal Fare,Non Stop,5955
2,Vistara,21:40:00,23:50:00,2h 10m,New Delhi,Mumbai,No Meal Fare,Non Stop,5955
3,Vistara,19:50:00,22:00:00,2h 10m,New Delhi,Mumbai,No Meal Fare,Non Stop,6060
4,Vistara,09:30:00,11:35:00,2h 05m,New Delhi,Mumbai,Free Meal,Non Stop,6375
...,...,...,...,...,...,...,...,...,...
3350,Air Asia,19:35:00,21:15:00,25h 40m,Jaipur,Mumbai,Free Meal,1 Stop,9311
3351,Air Asia,05:15:00,10:25:00,5h 10m,Jaipur,Chennai,eCash 250,1 Stop,10676
3352,Air Asia,14:20:00,20:25:00,6h 05m,Jaipur,Chennai,eCash 250,1 Stop,10676
3353,Air Asia,05:15:00,12:00:00,6h 45m,Jaipur,Chennai,eCash 250,1 Stop,10676


In [48]:
# saving the data into excel file
df.to_excel("Flight_Prices(web scraping).xlsx")

# successfuly save the data in excel file